In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=1,
        bias=False,
    )


def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=1,
        stride=stride,
        bias=False,
    )


In [1]:
class MyYolo(nn.Module):
    def __init__(
        self,
        device: torch.device,
        input_size: int = 416,
        num_classes: int = 20,
        trainable: bool = False,
        conf_thresh: float = 0.01,
        nms_thresh: float = 0.5,
    ):
        """
        device: cpu or cuda
        input_size: the size of the image that this model need to handle
        num_classes: the number of classes that this model need to handle
        trainable: whether this model is trainable. Set true in training, false in inference
        conf_thresh: the threshold of confidence
        nms_thresh: the threshold of non-maximum suppression (NMS)
        """

        super(MyYolo, self).__init__()
        self.device = device
        self.input_size = input_size
        self.num_classes = num_classes
        self.trainable = trainable
        self.conf_thresh = conf_thresh
        self.nms_thresh = nms_thresh
        self.grid_cell = self.create_grid(input_size)

        self.backbone = None
        self.neck = None
        self.head = None
        self.pred = nn.Conv2d(512, 1 + self.num_classes + 4, 1)

    def create_grid(self, input_size):
        ...

    def reset_grid(self, input_size):
        ...

    def decode_boxes(self, pred):
        ...

    def nms(self, dets, scores):
        x1 = dets[:, 0]
        y1 = dets[:, 1]
        x2 = dets[:, 2]
        y2 = dets[:, 3]

        areas = (x2 - x1 + 1) * (y2 - y1 + 1)
        order = scores.argsort()[::-1]

        keep = []
        while order.size > 0:
            i = order[0]
            keep.append(i)
            xx1 = np.maximum(x1[i], x1[order[1:]])
            yy1 = np.maximum(y1[i], y1[order[1:]])
            xx2 = np.minimum(x2[i], x2[order[1:]])
            yy2 = np.minimum(y2[i], y2[order[1:]])

            w = np.maximum(0.0, xx2 - xx1 + 1)
            h = np.maximum(0.0, yy2 - yy1 + 1)
            inter = w * h
            ovr = inter / (areas[i] + areas[order[1:]] - inter)

            inds = np.where(ovr <= self.nms_thresh)[0]
            order = order[inds + 1]
        return keep

    def forward(self, x):
        identity = x

        out = self.conv1(out)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity
        out = self.relu(out)

        return out


NameError: name 'nn' is not defined